In [ ]:
# =============================
# 0. 기본 셋업 & 패키지 설치
# =============================
!pip install timm iterative-stratification kagglehub grad-cam -q

import os, json, time, random, math, glob
from collections import defaultdict

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as tvm

import torchvision.transforms as T
import timm
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    average_precision_score
)

import matplotlib.pyplot as plt
import kagglehub

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [ ]:
# =============================
# 1. 데이터 다운로드 및 기본 메타 구성
# =============================

path = kagglehub.dataset_download("nih-chest-xrays/data")

# KaggleHub로 NIH Chest X-ray 다운로드
BASE_DIR = "/root/.cache/kagglehub/datasets/nih-chest-xrays/data/versions/3"
print("BASE_DIR:", BASE_DIR)

LABEL_CSV = os.path.join(BASE_DIR, "Data_Entry_2017.csv")
TRAIN_TXT = os.path.join(BASE_DIR, "train_val_list.txt")
TEST_TXT  = os.path.join(BASE_DIR, "test_list.txt")

df_meta = pd.read_csv(LABEL_CSV)

def parse_labels(s):
    parts = [p.strip() for p in s.split('|')]
    return [p for p in parts if p != '']

df_meta['labels_list'] = df_meta['Finding Labels'].apply(parse_labels)

all_labels = sorted({l for labels in df_meta['labels_list'] for l in labels})
label2idx = {l: i for i, l in enumerate(all_labels)}
idx2label = {i: l for l, i in label2idx.items()}
num_classes = len(all_labels)

print("Num classes:", num_classes)
print("Labels:", all_labels)


BASE_DIR: /root/.cache/kagglehub/datasets/nih-chest-xrays/data/versions/3
Num classes: 15
Labels: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [ ]:
# =============================
# 1-1. 이미지 경로 매핑 (파일명 → full path)
# NIH Chest 데이터는 images_001/... 등의 폴더 안에 존재
# =============================

image_paths = glob.glob(os.path.join(BASE_DIR, "images_*", "images", "*"))
print("Total image files found:", len(image_paths))

fname2path = {os.path.basename(p): p for p in image_paths}
list(fname2path.items())[:5]

# =============================
# 1-2. train/test 리스트 읽기
# =============================

def read_list_file(path):
    with open(path, 'r') as f:
        lines = [ln.strip() for ln in f.readlines()]
    files = [os.path.basename(ln) for ln in lines if ln.strip()]
    return files

train_val_files = read_list_file(TRAIN_TXT)  # 86524 개
test_files      = read_list_file(TEST_TXT)   # 25596 개

print("Train+Val files:", len(train_val_files))
print("Test files:", len(test_files))

meta_subset = df_meta[['Image Index', 'labels_list']].copy()
meta_subset.rename(columns={'Image Index': 'filename'}, inplace=True)
fname2labels = dict(zip(meta_subset['filename'], meta_subset['labels_list']))

# =============================
# 1-3. 파일 목록과 메타데이터 조인
# =============================

def labels_to_multihot(labels):
    vec = np.zeros(num_classes, dtype=np.float32)
    for l in labels:
        if l in label2idx:
            vec[label2idx[l]] = 1.0
    return vec

def make_df_from_files(file_list):
    rows = []
    for fn in file_list:
        if fn not in fname2labels:
            continue
        if fn not in fname2path:
            continue
        labels = fname2labels[fn]
        rows.append({
            "filename": fn,
            "path": fname2path[fn],
            "labels_list": labels,
            "y": labels_to_multihot(labels)
        })
    return pd.DataFrame(rows)

df_train_val_all = make_df_from_files(train_val_files)
df_test_all      = make_df_from_files(test_files)

print(df_train_val_all.shape, df_test_all.shape)
df_train_val_all.head()


Total image files found: 112120
Train+Val files: 86524
Test files: 25596
(86524, 4) (25596, 4)


,filename,path,labels_list,y
0,00000001_000.png,/root/.cache/kagglehub/datasets/nih-chest-xray...,[Cardiomegaly],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,00000001_001.png,/root/.cache/kagglehub/datasets/nih-chest-xray...,"[Cardiomegaly, Emphysema]","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,00000001_002.png,/root/.cache/kagglehub/datasets/nih-chest-xray...,"[Cardiomegaly, Effusion]","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,00000002_000.png,/root/.cache/kagglehub/datasets/nih-chest-xray...,[No Finding],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,00000004_000.png,/root/.cache/kagglehub/datasets/nih-chest-xray...,"[Mass, Nodule]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
# =============================
# 2. Multilabel stratified split (train 30%, val 10% of original train)
# 한 번 분할한 결과를 CSV로 저장/로드
# =============================

split_csv_path = os.path.join(BASE_DIR, "split_train_val_30_10.csv")

if os.path.exists(split_csv_path):
    print("Loading existing split:", split_csv_path)
    df_split = pd.read_csv(split_csv_path)
else:
    print("Creating new stratified split...")
    N = len(df_train_val_all)
    X_dummy = np.zeros((N, 1))
    Y       = np.stack(df_train_val_all['y'].values)

    indices = np.arange(N)

    # 1) 먼저 전체의 30%를 train, 나머지 70%를 remaining으로
    # test_size=None으로 설정하여 나머지로 자동 계산되게 함 (rounding issue 방지)
    msss1 = MultilabelStratifiedShuffleSplit(
        n_splits=1, train_size=0.3, test_size=None, random_state=42
    )
    train_idx, rem_idx = next(msss1.split(X_dummy, Y))

    # 2) remaining(70%)에서 다시 val 10% / 나머지 60%
    Y_rem = Y[rem_idx]
    X_dummy_rem = np.zeros((len(rem_idx), 1))

    # val 비율은 전체 대비 10%이므로, remaining 내에서는 10 / 70 ≈ 0.142857
    val_ratio_in_rem = 10.0 / 70.0

    # test_size=None으로 설정하여 rounding issue 방지
    msss2 = MultilabelStratifiedShuffleSplit(
        n_splits=1, train_size=val_ratio_in_rem, test_size=None,
        random_state=43
    )
    val_sub_idx, _ = next(msss2.split(X_dummy_rem, Y_rem))
    val_idx = rem_idx[val_sub_idx]

    split_labels = np.array(['unused'] * N, dtype=object)
    split_labels[train_idx] = 'train'
    split_labels[val_idx]   = 'val'

    df_split = df_train_val_all[['filename']].copy()
    df_split['split'] = split_labels
    df_split.to_csv(split_csv_path, index=False)
    print("Saved split to:", split_csv_path)

# df_split: filename, split
print(df_split['split'].value_counts())

# =============================
# 2-1. split 정보 병합해서 최종 train/val/test DF 구성
# =============================

df_train_val_all = df_train_val_all.merge(df_split, on='filename', how='left')
df_train = df_train_val_all[df_train_val_all['split'] == 'train'].reset_index(drop=True)
df_val   = df_train_val_all[df_train_val_all['split'] == 'val'].reset_index(drop=True)

# test는 전체 test_list 그대로 사용
df_test = df_test_all.reset_index(drop=True)

print("Train size:", len(df_train))
print("Val size:", len(df_val))
print("Test size:", len(df_test))

Loading existing split: /root/.cache/kagglehub/datasets/nih-chest-xrays/data/versions/3/split_train_val_30_10.csv
split
unused    51918
train     25974
val        8632
Name: count, dtype: int64
Train size: 25974
Val size: 8632
Test size: 25596


In [ ]:
# =============================
# 3. Dataset & DataLoader
# =============================

IMG_SIZE = 224

train_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

val_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

class MultiLabelDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df.reset_index(drop=True)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['path']).convert('RGB')
        if self.transforms:
            img = self.transforms(img)
        y = torch.from_numpy(row['y'])
        return img, y

train_ds = MultiLabelDataset(df_train, transforms=train_tfms)
val_ds   = MultiLabelDataset(df_val,   transforms=val_tfms)
test_ds  = MultiLabelDataset(df_test,  transforms=val_tfms)

BATCH_SIZE = 128

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4, pin_memory=True)

len(train_loader), len(val_loader), len(test_loader)



(203, 68, 200)

In [ ]:
# =============================
# 4. 모델 생성 (Backbone freeze + 마지막 layer만 학습)
# =============================

def create_backbone(model_name, pretrained=True):
    """
    model_name: 'vit', 'resnet50', 'efficientnet'
    torchvision의 ImageNet pretrained 모델 사용.
    backbone의 마지막 FC/classifier는 Identity로 바꾸고,
    그 전의 feature dimension을 in_features로 리턴.
    """
    if model_name == 'resnet50':
        weights = tvm.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = tvm.resnet50(weights=weights)
        in_features = backbone.fc.in_features
        backbone.fc = nn.Identity()

    elif model_name == 'efficientnet':
        weights = tvm.EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = tvm.efficientnet_b0(weights=weights)
        # classifier는 [Dropout, Linear] 구조
        in_features = backbone.classifier[1].in_features
        backbone.classifier = nn.Identity()

    elif model_name == 'vit':
        weights = tvm.ViT_B_16_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = tvm.vit_b_16(weights=weights)
        in_features = backbone.heads.head.in_features
        backbone.heads.head = nn.Identity()

    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    # backbone freeze
    for p in backbone.parameters():
        p.requires_grad = False

    return backbone, in_features


class LinearHeadModel(nn.Module):
    def __init__(self, backbone, in_features, num_classes):
        super().__init__()
        self.backbone = backbone
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        feats = self.backbone(x)          # (B, in_features)
        logits = self.classifier(feats)
        return logits

# supervised contrastive용: projection head까지 포함한 embedding 모델
class SupConModel(nn.Module):
    def __init__(self, backbone, in_features, proj_dim=128):
        super().__init__()
        self.backbone = backbone
        self.proj = nn.Linear(in_features, proj_dim)

    def forward(self, x):
        feats = self.backbone(x)
        z = self.proj(feats)
        z = nn.functional.normalize(z, dim=1)
        return z


In [ ]:
# =============================
# 5. Metric 계산 함수 (6개 지표)
# =============================

def compute_metrics(logits, targets, threshold=0.5):
    probs = 1 / (1 + np.exp(-logits))  # sigmoid
    preds = (probs >= threshold).astype(int)

    metrics = {}

    metrics['f1_micro'] = f1_score(targets, preds, average='micro', zero_division=0)
    metrics['f1_macro'] = f1_score(targets, preds, average='macro', zero_division=0)

    try:
        auc_macro = roc_auc_score(targets, probs, average='macro')
    except ValueError:
        auc_macro = np.nan
    metrics['auc_macro'] = float(auc_macro)

    ap_per_class = []
    for c in range(targets.shape[1]):
        if np.sum(targets[:, c]) == 0:
            continue
        ap = average_precision_score(targets[:, c], probs[:, c])
        ap_per_class.append(ap)
    metrics['mAP'] = float(np.mean(ap_per_class)) if len(ap_per_class) > 0 else np.nan

    acc_micro = (targets == preds).mean()
    metrics['acc_micro'] = float(acc_micro)

    acc_per_class = []
    for c in range(targets.shape[1]):
        acc_c = (targets[:, c] == preds[:, c]).mean()
        acc_per_class.append(acc_c)
    metrics['acc_macro'] = float(np.mean(acc_per_class))

    return metrics, probs, preds


In [ ]:
# =============================
# 6. 공통 train/eval 루프 (supervised BCE)
# =============================

def train_one_epoch_supervised(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)


def eval_supervised(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    all_logits = []
    all_targets = []
    with torch.no_grad():
        for images, targets in loader:
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            logits = model(images)
            loss = criterion(logits, targets)
            running_loss += loss.item() * images.size(0)

            all_logits.append(logits.cpu())
            all_targets.append(targets.cpu())
    avg_loss = running_loss / len(loader.dataset)
    all_logits = torch.cat(all_logits).numpy()
    all_targets = torch.cat(all_targets).numpy()
    return avg_loss, all_logits, all_targets


In [ ]:
# =============================
# 7. Supervised Contrastive Loss (multi-label)
# supCon: 두 샘플이 하나라도 같은 label을 공유하면 positive pair
# =============================

def supervised_contrastive_loss(features, labels, temperature=0.07):
    """
    features: (B, d), normalized
    labels: (B, C) multi-hot
    """
    device = features.device
    batch_size = features.shape[0]

    # similarity matrix
    sim_matrix = torch.matmul(features, features.T) / temperature  # (B,B)
    # 제거: self-similarity
    logits_mask = torch.ones_like(sim_matrix) - torch.eye(batch_size, device=device)
    sim_matrix = sim_matrix * logits_mask

    # positive mask: share at least one label
    labels = labels.float()
    # (B,B): entry (i,j) = 1 if share at least one label and i!=j
    pos_mask = (torch.matmul(labels, labels.T) > 0).float() * logits_mask

    # log-sum-exp over all j!=i
    exp_sim = torch.exp(sim_matrix) * logits_mask
    log_prob = sim_matrix - torch.log(exp_sim.sum(dim=1, keepdim=True) + 1e-8)

    # for each i, average over positive j
    pos_count = pos_mask.sum(dim=1)
    # 피하기: positive가 없는 샘플은 contribution 0
    mean_log_prob_pos = (pos_mask * log_prob).sum(dim=1) / (pos_count + 1e-8)

    # loss = - 평균(mean over i of mean_log_prob_pos)
    loss = -mean_log_prob_pos.mean()
    return loss

# =============================
# 7-1. SupCon 학습 루프 (backbone freeze + proj head만 학습)
# =============================

def train_one_epoch_supcon(model, loader, optimizer, temperature=0.07):
    model.train()
    running_loss = 0.0
    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad()
        z = model(images)  # normalized embeddings
        loss = supervised_contrastive_loss(z, targets, temperature=temperature)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)

# =============================
# 7-2. SupCon으로 backbone+proj 고정한 후,
#      같은 backbone의 feature를 사용해서 Linear classifier(BCE) 학습
# =============================

def extract_features(backbone, loader):
    backbone.eval()
    feats_list = []
    labels_list = []
    with torch.no_grad():
        for images, targets in loader:
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            feats = backbone(images)
            feats_list.append(feats.cpu())
            labels_list.append(targets.cpu())
    feats = torch.cat(feats_list)
    labels = torch.cat(labels_list)
    return feats, labels


In [ ]:
# =============================
# 8. 전체 실험 함수
# =============================

def run_supervised_experiment(model_name, num_epochs=10, lr=1e-3, save_dir="results"):
    os.makedirs(save_dir, exist_ok=True)

    backbone, in_features = create_backbone(model_name, pretrained=True)
    model = LinearHeadModel(backbone, in_features, num_classes).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr=lr)

    best_val_loss = float('inf')
    best_state = None

    history = {'epoch': [], 'train_loss': [], 'val_loss': []}

    for epoch in range(1, num_epochs+1):
        t0 = time.time()
        train_loss = train_one_epoch_supervised(model, train_loader, optimizer, criterion)
        val_loss, _, _ = eval_supervised(model, val_loader, criterion)
        elapsed = time.time() - t0

        history['epoch'].append(epoch)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        print(f"[Supervised-{model_name}] Epoch {epoch}/{num_epochs} "
              f"train={train_loss:.4f} val={val_loss:.4f} time={elapsed:.1f}s")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict().copy()

    # 기록 및 모델 저장
    hist_df = pd.DataFrame(history)
    hist_df.to_csv(os.path.join(save_dir, f"{model_name}_supervised_history.csv"), index=False)

    plt.figure()
    plt.plot(hist_df['epoch'], hist_df['train_loss'], label='train')
    plt.plot(hist_df['epoch'], hist_df['val_loss'], label='val')
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(f"{model_name} Supervised Loss")
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"{model_name}_supervised_loss.png"))
    plt.close()

    weight_path = os.path.join(save_dir, f"{model_name}_supervised_best.pth")
    if best_state is not None:
        torch.save(best_state, weight_path)
        model.load_state_dict(best_state)

    # test 평가
    test_loss, test_logits, test_targets = eval_supervised(model, test_loader, criterion)
    metrics, probs, preds = compute_metrics(test_logits, test_targets)

    print(f"[Supervised-{model_name}] Test: F1_micro={metrics['f1_micro']:.4f}, "
          f"F1_macro={metrics['f1_macro']:.4f}, AUC_macro={metrics['auc_macro']:.4f}, "
          f"mAP={metrics['mAP']:.4f}")

    # metrics 저장
    with open(os.path.join(save_dir, f"{model_name}_supervised_metrics.json"), "w") as f:
        json.dump(metrics, f, indent=2)

    # classification report 저장
    from sklearn.metrics import classification_report
    report = classification_report(
        test_targets, preds,
        target_names=[idx2label[i] for i in range(num_classes)],
        zero_division=0, output_dict=True
    )
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv(os.path.join(save_dir, f"{model_name}_supervised_classification_report.csv"))

    return metrics, weight_path


In [ ]:
def run_supcon_experiment(model_name, num_epochs_supcon=5, num_epochs_cls=5, lr=1e-3, save_dir="results"):
    os.makedirs(save_dir, exist_ok=True)

    # 1) backbone + projection head로 SupCon 학습
    backbone, in_features = create_backbone(model_name, pretrained=True)
    supcon_model = SupConModel(backbone, in_features, proj_dim=128).to(device)

    # proj head만 학습 (backbone은 이미 requires_grad=False)
    optimizer = torch.optim.Adam(supcon_model.proj.parameters(), lr=lr)

    history_supcon = {'epoch': [], 'train_loss': []}
    for epoch in range(1, num_epochs_supcon+1):
        t0 = time.time()
        train_loss = train_one_epoch_supcon(supcon_model, train_loader, optimizer, temperature=0.07)
        elapsed = time.time() - t0
        history_supcon['epoch'].append(epoch)
        history_supcon['train_loss'].append(train_loss)

        print(f"[SupCon-{model_name}] Epoch {epoch}/{num_epochs_supcon} "
              f"train_supcon_loss={train_loss:.4f} time={elapsed:.1f}s")

    hist_df = pd.DataFrame(history_supcon)
    hist_df.to_csv(os.path.join(save_dir, f"{model_name}_supcon_pretrain_history.csv"), index=False)

    plt.figure()
    plt.plot(hist_df['epoch'], hist_df['train_loss'], label='supcon_train_loss')
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.title(f"{model_name} SupCon Pretrain Loss")
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"{model_name}_supcon_pretrain_loss.png"))
    plt.close()

    # 2) SupCon으로 학습된 backbone을 그대로 쓰고 (proj는 feature용),
    #    Linear classifier를 따로 학습
    #    여기서는 backbone 출력 feature 위에 classifier를 붙인다.
    backbone_after = supcon_model.backbone  # 이미 SupCon에서 backbone은 freeze

    cls_model = LinearHeadModel(backbone_after, in_features, num_classes).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer_cls = torch.optim.Adam(cls_model.classifier.parameters(), lr=lr)

    best_val_loss = float('inf')
    best_state = None
    history_cls = {'epoch': [], 'train_loss': [], 'val_loss': []}

    for epoch in range(1, num_epochs_cls+1):
        t0 = time.time()
        train_loss = train_one_epoch_supervised(cls_model, train_loader, optimizer_cls, criterion)
        val_loss, _, _ = eval_supervised(cls_model, val_loader, criterion)
        elapsed = time.time() - t0

        history_cls['epoch'].append(epoch)
        history_cls['train_loss'].append(train_loss)
        history_cls['val_loss'].append(val_loss)

        print(f"[SupCon-CLS-{model_name}] Epoch {epoch}/{num_epochs_cls} "
              f"train={train_loss:.4f} val={val_loss:.4f} time={elapsed:.1f}s")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = cls_model.state_dict().copy()

    hist_df2 = pd.DataFrame(history_cls)
    hist_df2.to_csv(os.path.join(save_dir, f"{model_name}_supcon_cls_history.csv"), index=False)

    plt.figure()
    plt.plot(hist_df2['epoch'], hist_df2['train_loss'], label='train')
    plt.plot(hist_df2['epoch'], hist_df2['val_loss'], label='val')
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.title(f"{model_name} SupCon+CLS Loss")
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"{model_name}_supcon_cls_loss.png"))
    plt.close()

    weight_path = os.path.join(save_dir, f"{model_name}_supcon_best.pth")
    if best_state is not None:
        torch.save(best_state, weight_path)
        cls_model.load_state_dict(best_state)

    # test 평가
    test_loss, test_logits, test_targets = eval_supervised(cls_model, test_loader, criterion)
    metrics, probs, preds = compute_metrics(test_logits, test_targets)

    print(f"[SupCon-{model_name}] Test: F1_micro={metrics['f1_micro']:.4f}, "
          f"F1_macro={metrics['f1_macro']:.4f}, AUC_macro={metrics['auc_macro']:.4f}, "
          f"mAP={metrics['mAP']:.4f}")

    with open(os.path.join(save_dir, f"{model_name}_supcon_metrics.json"), "w") as f:
        json.dump(metrics, f, indent=2)

    from sklearn.metrics import classification_report
    report = classification_report(
        test_targets, preds,
        target_names=[idx2label[i] for i in range(num_classes)],
        zero_division=0, output_dict=True
    )
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv(os.path.join(save_dir, f"{model_name}_supcon_classification_report.csv"))

    return metrics, weight_path


In [ ]:
# =============================
# 9. 전체 모델에 대해 실험 실행
# =============================

save_dir = "results_nih_chest"
os.makedirs(save_dir, exist_ok=True)

supervised_results = {}
supcon_results = {}

for name in ['vit', 'resnet50', 'efficientnet']:
    print(f"\n===== Supervised training for {name} =====")
    sup_metrics, sup_wpath = run_supervised_experiment(
        name, num_epochs=10, lr=1e-3, save_dir=save_dir
    )
    supervised_results[name] = sup_metrics

    print(f"\n===== Supervised Contrastive training for {name} =====")
    sc_metrics, sc_wpath = run_supcon_experiment(
        name, num_epochs_supcon=10, num_epochs_cls=10, lr=1e-3, save_dir=save_dir
    )
    supcon_results[name] = sc_metrics

# 결과 비교 테이블
supervised_df = pd.DataFrame(supervised_results).T
supcon_df     = pd.DataFrame(supcon_results).T

supervised_df.to_csv(os.path.join(save_dir, "supervised_model_comparison.csv"))
supcon_df.to_csv(os.path.join(save_dir, "supcon_model_comparison.csv"))

print("\nSupervised results:")
display(supervised_df)

print("\nSupCon results:")
display(supcon_df)



===== Supervised training for vit =====
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:02<00:00, 155MB/s]


[Supervised-vit] Epoch 1/10 train=0.1968 val=0.1830 time=185.8s
[Supervised-vit] Epoch 2/10 train=0.1806 val=0.1812 time=185.3s
[Supervised-vit] Epoch 3/10 train=0.1777 val=0.1790 time=184.9s
[Supervised-vit] Epoch 4/10 train=0.1762 val=0.1780 time=185.7s
[Supervised-vit] Epoch 5/10 train=0.1749 val=0.1785 time=185.3s
[Supervised-vit] Epoch 6/10 train=0.1741 val=0.1781 time=184.9s
[Supervised-vit] Epoch 7/10 train=0.1732 val=0.1782 time=185.0s
[Supervised-vit] Epoch 8/10 train=0.1726 val=0.1780 time=185.5s
[Supervised-vit] Epoch 9/10 train=0.1721 val=0.1776 time=185.5s
[Supervised-vit] Epoch 10/10 train=0.1714 val=0.1774 time=185.2s
[Supervised-vit] Test: F1_micro=0.2784, F1_macro=0.0725, AUC_macro=0.7170, mAP=0.1882

===== Supervised Contrastive training for vit =====
[SupCon-vit] Epoch 1/10 train_supcon_loss=4.8182 time=137.9s
[SupCon-vit] Epoch 2/10 train_supcon_loss=4.8049 time=138.3s
[SupCon-vit] Epoch 3/10 train_supcon_loss=4.8027 time=138.5s
[SupCon-vit] Epoch 4/10 train_supcon_

100%|██████████| 97.8M/97.8M [00:00<00:00, 189MB/s]


[Supervised-resnet50] Epoch 1/10 train=0.1987 val=0.1871 time=183.8s
[Supervised-resnet50] Epoch 2/10 train=0.1862 val=0.1851 time=183.6s
[Supervised-resnet50] Epoch 3/10 train=0.1841 val=0.1849 time=183.0s
[Supervised-resnet50] Epoch 4/10 train=0.1830 val=0.1856 time=183.2s
[Supervised-resnet50] Epoch 5/10 train=0.1821 val=0.1836 time=182.8s
[Supervised-resnet50] Epoch 6/10 train=0.1817 val=0.1848 time=183.3s
[Supervised-resnet50] Epoch 7/10 train=0.1808 val=0.1844 time=184.4s
[Supervised-resnet50] Epoch 8/10 train=0.1805 val=0.1840 time=184.2s
[Supervised-resnet50] Epoch 9/10 train=0.1794 val=0.1853 time=183.3s
[Supervised-resnet50] Epoch 10/10 train=0.1791 val=0.1840 time=183.9s
[Supervised-resnet50] Test: F1_micro=0.1970, F1_macro=0.0474, AUC_macro=0.6783, mAP=0.1632

===== Supervised Contrastive training for resnet50 =====
[SupCon-resnet50] Epoch 1/10 train_supcon_loss=4.8179 time=137.3s
[SupCon-resnet50] Epoch 2/10 train_supcon_loss=4.8126 time=137.0s
[SupCon-resnet50] Epoch 3/10

100%|██████████| 20.5M/20.5M [00:00<00:00, 129MB/s] 


[Supervised-efficientnet] Epoch 1/10 train=0.2207 val=0.1953 time=184.4s
[Supervised-efficientnet] Epoch 2/10 train=0.1971 val=0.1909 time=183.7s
[Supervised-efficientnet] Epoch 3/10 train=0.1928 val=0.1889 time=182.7s
[Supervised-efficientnet] Epoch 4/10 train=0.1898 val=0.1877 time=183.1s
[Supervised-efficientnet] Epoch 5/10 train=0.1880 val=0.1876 time=183.0s
[Supervised-efficientnet] Epoch 6/10 train=0.1864 val=0.1867 time=182.5s
[Supervised-efficientnet] Epoch 7/10 train=0.1844 val=0.1862 time=182.7s
[Supervised-efficientnet] Epoch 8/10 train=0.1830 val=0.1860 time=183.1s
[Supervised-efficientnet] Epoch 9/10 train=0.1827 val=0.1863 time=182.4s
[Supervised-efficientnet] Epoch 10/10 train=0.1818 val=0.1867 time=182.7s
[Supervised-efficientnet] Test: F1_micro=0.2613, F1_macro=0.0521, AUC_macro=0.6494, mAP=0.1550

===== Supervised Contrastive training for efficientnet =====
[SupCon-efficientnet] Epoch 1/10 train_supcon_loss=4.8389 time=137.8s
[SupCon-efficientnet] Epoch 2/10 train_sup